In [1]:
import pandas as pd

In [6]:
# !pip install html5lib

In [20]:
# tables = pd.read_html("https://trac.syr.edu/phptools/immigration/asyfile/")

I tried this read_html method, but it says "no table found"

In [7]:
import requests
from bs4 import BeautifulSoup

In [ ]:
# my_url = "https://trac.syr.edu/phptools/immigration/asyfile/"
# raw_html = requests.get(my_url).content

I tried a normal requests but the info isn't there

Resorting to Playwright

In [21]:
from playwright.async_api import async_playwright

In [67]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()

In [68]:
await page.goto("https://trac.syr.edu/phptools/immigration/asyfile/")

<Response url='https://trac.syr.edu/phptools/immigration/asyfile/' request=<Request url='https://trac.syr.edu/phptools/immigration/asyfile/' method='GET'>>

In [69]:
await page.get_by_role("button", name='Immigration Court State').click()
await page.get_by_role("option", name='Fiscal Year Application Filed').click()

In [70]:
await page.get_by_role("button", name='Custody').click()
await page.get_by_role("option", name='Month and Year Application Filed').click()

In [71]:
await page.get_by_role("button", name='Represented').click()
await page.get_by_role("option", name='Outcome').click()

In [72]:
html = await page.content()
html

'<html lang="en"><head>\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Immigration Court Asylum Filings</title>\n    <link href="/css/styles.css" rel="stylesheet">\n    <script src="https://unpkg.com/vue@3/dist/vue.global.prod.js"></script>\n    <script type="module" src="/immigration/js/hamburger.js"></script>\n    <script type="module" src="/immigration/js/search.js"></script>\n    <!--[-->\n    <!--]-->\n    <!--[-->\n    <script src="//d3js.org/d3.v3.min.js"></script>\n    <!--]-->\n    <!--[-->\n    <script type="module" src="asyfile.js"></script>\n    <!--]-->\n  </head>\n  <body class="bg-yellow-50">\n    <div class="bg-yellow-100 shadow-lg">\n      <div id="t-inner" class="max-w-7xl mx-auto">\n        <!-- Logo -->\n        <div id="t-logo" class="flex flex-row bg-yellow-100 justify-between pt-1 px-2 pb-0">\n          <div class="flex">\n            <div id="hamburger" class="p-4 pt-8 pb-2 lg:pl-20 pr-1" data-

In [74]:
soup_doc = BeautifulSoup(html, 'html.parser')

print(soup_doc.prettify())

<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <title>
   Immigration Court Asylum Filings
  </title>
  <link href="/css/styles.css" rel="stylesheet"/>
  <script src="https://unpkg.com/vue@3/dist/vue.global.prod.js">
  </script>
  <script src="/immigration/js/hamburger.js" type="module">
  </script>
  <script src="/immigration/js/search.js" type="module">
  </script>
  <!--[-->
  <!--]-->
  <!--[-->
  <script src="//d3js.org/d3.v3.min.js">
  </script>
  <!--]-->
  <!--[-->
  <script src="asyfile.js" type="module">
  </script>
  <!--]-->
 </head>
 <body class="bg-yellow-50">
  <div class="bg-yellow-100 shadow-lg">
   <div class="max-w-7xl mx-auto" id="t-inner">
    <!-- Logo -->
    <div class="flex flex-row bg-yellow-100 justify-between pt-1 px-2 pb-0" id="t-logo">
     <div class="flex">
      <div class="p-4 pt-8 pb-2 lg:pl-20 pr-1" data-v-app="" id="hamburger">
       <div class="text-right top-6">
     

In [76]:
tables = pd.read_html(html)

len(tables)

/var/folders/29/tm85f6r96dl6jl70q7g6h2rr0000gn/T/ipykernel_2977/899423628.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


3

In [90]:
years = tables[0].rename(columns={
    'Unnamed: 0': 'year',
    'Unnamed: 1': 'number of cases'
}).sort_values(by='year', ascending=False)

years

,year,number of cases
0,All,1957146
12,2023,58686
1,2022,256241
6,2021,92130
3,2020,197833
2,2019,216936
4,2018,166394
5,2017,146631
7,2016,83919
9,2015,64664


In [91]:
months = tables[1].rename(columns={
    'Unnamed: 0': 'year-month',
    'Unnamed: 1': 'number of cases'
}).sort_values(by='year-month', ascending=False)

months

,year-month,number of cases
0,All-2022,256241
2,2022-09,30214
1,2022-08,33637
5,2022-07,29145
4,2022-06,29223
6,2022-05,26061
3,2022-04,29417
7,2022-03,22420
8,2022-02,14043
9,2022-01,11552


In [88]:
outcomes = tables[2].rename(columns={
    'Unnamed: 0': 'outcome',
    'Unnamed: 1': 'number of cases'
})

outcomes

,outcome,number of cases
0,"All-2022, 2022-08",33637
1,Pending,31863
2,Removal Order,980
3,Other Closure,421
4,Grant Relief,214
5,Terminate Proceedings,85
6,Voluntary Departure,68
7,Pros. Discretion,6


## Trying out API

### year-month

In [11]:
# for fiscal year 2022

url = "https://trac.syr.edu/phptools/immigration/asyfile/graph.php?stat=cls&timescale=fymon&fy=1&timeunit=number"

In [12]:
import requests
response = requests.get(url)
response

<Response [200]>

In [13]:
yr2022 = response.json()
yr2022

{'title': '2022',
 'x_axis_label': 'Month and Year Application Filed',
 'y_axis_label': {'number': 'Count', 'percent': 'Percent'},
 'timeline': [{'fymon': '2021-10', 'number': '8995', 'percent': '100.0000'},
  {'fymon': '2021-11', 'number': '10297', 'percent': '100.0000'},
  {'fymon': '2021-12', 'number': '11237', 'percent': '100.0000'},
  {'fymon': '2022-01', 'number': '11552', 'percent': '100.0000'},
  {'fymon': '2022-02', 'number': '14043', 'percent': '100.0000'},
  {'fymon': '2022-03', 'number': '22420', 'percent': '100.0000'},
  {'fymon': '2022-04', 'number': '29417', 'percent': '100.0000'},
  {'fymon': '2022-05', 'number': '26061', 'percent': '100.0000'},
  {'fymon': '2022-06', 'number': '29223', 'percent': '100.0000'},
  {'fymon': '2022-07', 'number': '29145', 'percent': '100.0000'},
  {'fymon': '2022-08', 'number': '33637', 'percent': '100.0000'},
  {'fymon': '2022-09', 'number': '30214', 'percent': '100.0000'}]}

In [14]:
yr2022.keys()

dict_keys(['title', 'x_axis_label', 'y_axis_label', 'timeline'])

In [15]:
yr2022['timeline']

[{'fymon': '2021-10', 'number': '8995', 'percent': '100.0000'},
 {'fymon': '2021-11', 'number': '10297', 'percent': '100.0000'},
 {'fymon': '2021-12', 'number': '11237', 'percent': '100.0000'},
 {'fymon': '2022-01', 'number': '11552', 'percent': '100.0000'},
 {'fymon': '2022-02', 'number': '14043', 'percent': '100.0000'},
 {'fymon': '2022-03', 'number': '22420', 'percent': '100.0000'},
 {'fymon': '2022-04', 'number': '29417', 'percent': '100.0000'},
 {'fymon': '2022-05', 'number': '26061', 'percent': '100.0000'},
 {'fymon': '2022-06', 'number': '29223', 'percent': '100.0000'},
 {'fymon': '2022-07', 'number': '29145', 'percent': '100.0000'},
 {'fymon': '2022-08', 'number': '33637', 'percent': '100.0000'},
 {'fymon': '2022-09', 'number': '30214', 'percent': '100.0000'}]

### removal order

In [7]:
# outcome across all year-month

url = "https://trac.syr.edu/phptools/immigration/asyfile/graph.php?stat=cls&timescale=fymon&outcome=3&timeunit=number"

In [8]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

response = requests.get(url)

In [10]:
removal_order = response.json()
removal_order

{'title': 'Removal Order',
 'x_axis_label': 'Month and Year Application Filed',
 'y_axis_label': {'number': 'Count', 'percent': 'Percent'},
 'timeline': [{'fymon': '2000-10', 'number': '1932', 'percent': '40.2836'},
  {'fymon': '2000-11', 'number': '1821', 'percent': '41.5848'},
  {'fymon': '2000-12', 'number': '1589', 'percent': '39.9748'},
  {'fymon': '2001-01', 'number': '1744', 'percent': '41.0160'},
  {'fymon': '2001-02', 'number': '1731', 'percent': '41.1555'},
  {'fymon': '2001-03', 'number': '2021', 'percent': '38.1969'},
  {'fymon': '2001-04', 'number': '2002', 'percent': '38.5222'},
  {'fymon': '2001-05', 'number': '2185', 'percent': '38.2327'},
  {'fymon': '2001-06', 'number': '1902', 'percent': '33.6697'},
  {'fymon': '2001-07', 'number': '1890', 'percent': '37.6194'},
  {'fymon': '2001-08', 'number': '2256', 'percent': '33.1132'},
  {'fymon': '2001-09', 'number': '1474', 'percent': '35.9249'},
  {'fymon': '2001-10', 'number': '2192', 'percent': '40.9490'},
  {'fymon': '200

### pending

In [29]:
url = "https://trac.syr.edu/phptools/immigration/asyfile/graph.php?stat=cls&timescale=fymon&outcome=2&timeunit=number"

In [30]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

response = requests.get(url)

In [31]:
pending = response.json()
pending

{'title': 'Pending',
 'x_axis_label': 'Month and Year Application Filed',
 'y_axis_label': {'number': 'Count', 'percent': 'Percent'},
 'timeline': [{'fymon': '2000-10', 'number': '21', 'percent': '0.4379'},
  {'fymon': '2000-11', 'number': '23', 'percent': '0.5252'},
  {'fymon': '2000-12', 'number': '20', 'percent': '0.5031'},
  {'fymon': '2001-01', 'number': '18', 'percent': '0.4233'},
  {'fymon': '2001-02', 'number': '12', 'percent': '0.2853'},
  {'fymon': '2001-03', 'number': '21', 'percent': '0.3969'},
  {'fymon': '2001-04', 'number': '35', 'percent': '0.6735'},
  {'fymon': '2001-05', 'number': '16', 'percent': '0.2800'},
  {'fymon': '2001-06', 'number': '13', 'percent': '0.2301'},
  {'fymon': '2001-07', 'number': '8', 'percent': '0.1592'},
  {'fymon': '2001-08', 'number': '34', 'percent': '0.4990'},
  {'fymon': '2001-09', 'number': '9', 'percent': '0.2194'},
  {'fymon': '2001-10', 'number': '15', 'percent': '0.2802'},
  {'fymon': '2001-11', 'number': '26', 'percent': '0.4829'},
  

### granted relief

In [36]:
url = "https://trac.syr.edu/phptools/immigration/asyfile/graph.php?stat=cls&timescale=fymon&outcome=1&timeunit=number"

In [37]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

response = requests.get(url)

In [38]:
granted = response.json()
granted

{'title': 'Grant Relief',
 'x_axis_label': 'Month and Year Application Filed',
 'y_axis_label': {'number': 'Count', 'percent': 'Percent'},
 'timeline': [{'fymon': '2000-10', 'number': '1617', 'percent': '33.7156'},
  {'fymon': '2000-11', 'number': '1513', 'percent': '34.5513'},
  {'fymon': '2000-12', 'number': '1454', 'percent': '36.5786'},
  {'fymon': '2001-01', 'number': '1591', 'percent': '37.4177'},
  {'fymon': '2001-02', 'number': '1449', 'percent': '34.4508'},
  {'fymon': '2001-03', 'number': '1886', 'percent': '35.6454'},
  {'fymon': '2001-04', 'number': '1939', 'percent': '37.3100'},
  {'fymon': '2001-05', 'number': '1982', 'percent': '34.6807'},
  {'fymon': '2001-06', 'number': '1927', 'percent': '34.1122'},
  {'fymon': '2001-07', 'number': '1757', 'percent': '34.9721'},
  {'fymon': '2001-08', 'number': '2356', 'percent': '34.5809'},
  {'fymon': '2001-09', 'number': '1457', 'percent': '35.5106'},
  {'fymon': '2001-10', 'number': '1834', 'percent': '34.2612'},
  {'fymon': '2001

### other closure

In [74]:
url = "https://trac.syr.edu/phptools/immigration/asyfile/graph.php?stat=cls&timescale=fymon&outcome=4&timeunit=number"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

response = requests.get(url)
other_closure = response.json()
df_other_closure = pd.json_normalize(other_closure['timeline'])
df_other_closure.to_csv('other_closure.csv', index=False)

### terminate proceedings

In [73]:
url = "https://trac.syr.edu/phptools/immigration/asyfile/graph.php?stat=cls&timescale=fymon&outcome=5&timeunit=number"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

response = requests.get(url)
terminate = response.json()
df_terminate = pd.json_normalize(terminate['timeline'])
df_terminate.to_csv('terminate.csv', index=False)

#### Prosecutorial discretion (PD) is a power of immigration authorities to stop working on a deportation case, which can be beneficial for some asylum cases.

In [72]:
url = "https://trac.syr.edu/phptools/immigration/asyfile/graph.php?stat=cls&timescale=fymon&outcome=6&timeunit=number"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

response = requests.get(url)
pros_discretion = response.json()
df_pros_discretion = pd.json_normalize(pros_discretion['timeline'])
df_pros_discretion.to_csv('pros_discretion.csv', index=False)

### voluntary departure

In [71]:
url = "https://trac.syr.edu/phptools/immigration/asyfile/graph.php?stat=cls&timescale=fymon&outcome=7&timeunit=number"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

response = requests.get(url)
volunt_depart = response.json()
df_volunt_depart = pd.json_normalize(volunt_depart['timeline'])
df_volunt_depart.to_csv('voluntary_departure.csv', index=False)

In [48]:
import pandas as pd

df2022 = pd.json_normalize(yr2022['timeline'])

df2022 = df2022.drop(columns=['percent'])

df2022

,fymon,number
0,2021-10,8995
1,2021-11,10297
2,2021-12,11237
3,2022-01,11552
4,2022-02,14043
5,2022-03,22420
6,2022-04,29417
7,2022-05,26061
8,2022-06,29223
9,2022-07,29145


In [20]:
df2022.to_csv('yr2022.csv', index=False)

In [21]:
df_removal = pd.json_normalize(removal_order['timeline'])
df_removal

,fymon,number,percent
0,2000-10,1932,40.2836
1,2000-11,1821,41.5848
2,2000-12,1589,39.9748
3,2001-01,1744,41.0160
4,2001-02,1731,41.1555
...,...,...,...
261,2022-07,1354,4.6457
262,2022-08,980,2.9135
263,2022-09,671,2.2208
264,2022-10,337,1.0945


In [35]:
df_removal.to_csv('removal.csv', index=False)

In [33]:
df_pending = pd.json_normalize(pending['timeline'])
df_pending

,fymon,number,percent
0,2000-10,21,0.4379
1,2000-11,23,0.5252
2,2000-12,20,0.5031
3,2001-01,18,0.4233
4,2001-02,12,0.2853
...,...,...,...
261,2022-07,26861,92.1633
262,2022-08,31863,94.7260
263,2022-09,29035,96.0978
264,2022-10,30095,97.7396


In [34]:
df_pending.to_csv('pending.csv', index=False)

In [40]:
df_granted = pd.json_normalize(granted['timeline'])
df_granted

,fymon,number,percent
0,2000-10,1617,33.7156
1,2000-11,1513,34.5513
2,2000-12,1454,36.5786
3,2001-01,1591,37.4177
4,2001-02,1449,34.4508
...,...,...,...
261,2022-07,332,1.1391
262,2022-08,214,0.6362
263,2022-09,103,0.3409
264,2022-10,77,0.2501


In [41]:
df_granted.to_csv('granted.csv', index=False)

In [70]:
# without the for loop

# outcome2022 = pd.merge(df2022, df_removal[['fymon', 'number']], on='fymon', how='left', suffixes=('', '_removal'))
# outcome2022 = pd.merge(outcome2022, df_pending[['fymon', 'number']], on='fymon', how='left', suffixes=('', '_pending'))
# outcome2022 = pd.merge(outcome2022, df_granted[['fymon', 'number']], on='fymon', how='left', suffixes=('', '_granted'))
# outcome2022 = pd.merge(outcome2022, df_other_closure[['fymon', 'number']], on='fymon', how='left', suffixes=('', '_other_closure'))
# outcome2022 = pd.merge(outcome2022, df_terminate[['fymon', 'number']], on='fymon', how='left', suffixes=('', '_terminate'))
# outcome2022 = pd.merge(outcome2022, df_pros_discretion[['fymon', 'number']], on='fymon', how='left', suffixes=('', '_pros_discretion'))
# outcome2022 = pd.merge(outcome2022, df_volunt_depart[['fymon', 'number']], on='fymon', how='left', suffixes=('', '_volunt_depart'))

### All outcomes in fiscal year 2022

In [77]:
# List of DataFrames and their names
dfs_to_merge = [
    ('pending', df_pending),
    ('removal', df_removal),
    ('granted', df_granted),
    ('other_closure', df_other_closure),
    ('terminate', df_terminate),
    ('volunt_depart', df_volunt_depart),
    ('pros_discretion', df_pros_discretion)
]

# Start with the main DataFrame
outcomes2022 = df2022

# Loop to merge each DataFrame
for name, df in dfs_to_merge:
    outcomes2022 = pd.merge(
        outcomes2022,
        df[['fymon', 'number']].rename(columns={'number': name}),
        on='fymon',
        how='left'
    )

# Renaming yr2022's 'number' column to 'total'
outcomes2022 = outcomes2022.rename(columns={'number': 'total'})

outcomes2022.to_csv('outcomes2022.csv', index=False)
outcomes2022

,fymon,total,pending,removal,granted,other_closure,terminate,volunt_depart,pros_discretion
0,2021-10,8995,6992,903,388,532,136,37,7
1,2021-11,10297,7673,1209,623,619,126,31,16
2,2021-12,11237,8469,1280,646,651,122,64,5
3,2022-01,11552,8824,1232,615,679,140,37,25
4,2022-02,14043,11114,1326,705,724,109,53,12
5,2022-03,22420,18409,1867,764,1086,221,62,11
6,2022-04,29417,25073,1779,782,1443,242,81,17
7,2022-05,26061,22471,1953,557,874,116,79,11
8,2022-06,29223,26165,1680,437,721,124,83,13
9,2022-07,29145,26861,1354,332,484,60,48,6
